In [ ]:
import csv
import os
import speech_recognition as sr
import gradio as gr
import threading
import time
import pyttsx3
import tempfile


In [ ]:

# Initialize the TTS engine
engine = pyttsx3.init()

# Load inventory from CSV
def load_inventory(file_path):
    inventory = {}
    try:
        with open(file_path, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                product = row["product"]
                inventory[product] = {"quantity": int(row["quantity"]), "price": float(row["price"])}
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error loading inventory: {str(e)}")
    return inventory

# Update the file path to the resources folder
csv_file_path = os.path.join('Resources', 'mock_inventory.csv')
inventory = load_inventory(csv_file_path)


In [ ]:

# Function to process commands
def process_command(command):
    tokens = command.lower().split()
    if "find" in tokens and "product" in tokens:
        return "Sure, I can help you find a product. What are you looking for?"
    else:
        return "I'm sorry, I didn't understand that. Can you please repeat?"

# Function to convert speech to text
# Microphone - where is audio coming from?
# with sr.Microphone() as source:
# def speech_to_text(audio):
#     recognizer = sr.Recognizer()
#     try:
#         with sr.AudioFile(audio) as source:
#             audio_data = recognizer.record(source)
#         text = recognizer.recognize_google(audio_data)
#         return text
#     except sr.UnknownValueError:
#         return "Sorry, I could not understand the audio."
#     except sr.RequestError as e:
#         return f"Could not request results; {e}"

def listen_and_recognize(use_microphone=True, audio_file=None):
    recognizer = sr.Recognizer()

    if use_microphone:
        with sr.Microphone() as source:
            print("Listening...")
            try:
                audio = recognizer.listen(source)
            except sr.WaitTimeoutError:
                print("Listening timed out. No speech detected.")
                return ""
    elif audio_file:
        if not isinstance(audio_file, str):
            print("Error: audio_file must be a string representing the file path.")
            return ""
        try:
            with sr.AudioFile(audio_file) as source:
                audio = recognizer.record(source)
        except FileNotFoundError:
            print(f"Error: File not found at {audio_file}")
            return ""
    else:
        print("Error: Please specify either use_microphone=True or provide an audio_file.")
        return ""

    try:
        text = recognizer.recognize_google(audio)
        print(f"Recognized: {text}")
        return text
    except sr.UnknownValueError:
        print("Sorry, I could not understand the audio.")
        return ""
    except sr.RequestError as e:
        print(f"Could not request results; {e}")
        return ""

# Function to convert text to speech
def text_to_speech(text):
    with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as temp_file:
        engine.save_to_file(text, temp_file.name)
        engine.runAndWait()
    return temp_file.name

def speak(text):
    # Creates a gTTS (Google Text-to-Speech) instance with provided text
    tts = gTTS(text=text, lang='en')
    # Define the filename for the audio file
    filename = "response.mp3"
    # Save to mp3 file
    tts.save(filename)
    # play the mp3 file to speak the captured text
    playsound.playsound(filename)

# Function to handle voice interface
def voice_interface(audio):
    if audio is not None:
        text = speech_to_text(audio)
        response = process_command(text)
        response_audio = text_to_speech(response)
        return f"You said: {text}\nResponse: {response}", response_audio
    else:
        return "No audio input received.", None


In [ ]:

# Function to play initial greeting
def play_initial_greeting():
    greeting_text = "Hi, I am Paxil Voice Assistant, I am listening."
    engine.say(greeting_text)
    engine.runAndWait()

# Launch the Gradio interface
interface = gr.Interface(
    fn=voice_interface,
    inputs=gr.Microphone(),
    outputs=["text", gr.Audio(type="filepath")],
    title="Paxil's Voice Assistant",
    description="Speak into the microphone and get a voice response!",
    live=True
)

# Play the initial greeting
# play_initial_greeting()

interface.launch(share=True)

play_initial_greeting()

print("Gradio interface launched. You can now interact with the Paxil's voice assistant.")